### Do số lượng dữ liệu khá nhiều, nên cần lọc lại, chỉ lấy các bình luận có nhiều từ -> Để giảm số chiều dữ liệu lại
- Giảm bớt dữ liệu 
- Giảm chiều đầu vào, tức giảm số từ
- Tiến hành chuyển token 
- Trích đặc trưng bằng bert

# Import

In [1]:
import numpy as np
import pandas as pd
pd.set_option("max_colwidth", 100)
import matplotlib.pyplot as plt


# Định nghĩa các biến
- max_len là chiều dài của input, dùng ở padding, tức mỗi câu phải có tối thiểu 20 ký tự, giả sử câu có 4 ký tự thì các số 0 sẽ được thêm vào cho đủ
- max_data_good tức số lượng bình luận tích cực được lấy
- max_data_bad tức số lượng bình luận tiêu cực
- min_words dùng để lọc lại các comment có 4 từ trở lên
- Do PhoBert trích đặc trưng ra khá lớn 768 chiều đc trích xuất cho mỗi câu, nên dung lượng Ram ko đủ đáp ứng, vì vậy phải giảm bớt số lượng dữ liệu 

In [2]:
# Max lenght vector
max_len = 20
# len dataset
max_data_good = 8000
max_data_bad = 6000
# index user random
index = 50
# number words
min_words = 4

# Load file

In [3]:
path_file = "./data/non_stop_word(3).json"
df = pd.read_json(path_file)

# Số lượng bình luận theo rate

In [4]:
print(len(df[df['rate']==0]))
print(len(df[df['rate']==1]))

1652191
306361


# Lọc bình luận tích cực
- Giảm bớt số lượng bình luận tích cực, bằng cách chỉ giữ lại nhừng bình luận có trên n từ trở lên

In [5]:
df_good = df[df['rate']==0]
def zero(text):
    arr = text.split()
    if (len(arr) >= min_words):
        return text
    else:
        return str('non')
df_good['content'] = df_good['content'].apply(lambda x: x.replace(
    str(x), zero(x)))

C:\Users\OnKhang\AppData\Local\Temp\ipykernel_5516\3547707240.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_good['content'] = df_good['content'].apply(lambda x: x.replace(


#### Số bình luận ít hơn max_words từ

In [6]:
len(df_good[df_good['content']=="non"])

526628

#### Số bình luận còn lại sau khi lọc

In [7]:
df_good_new = df_good[df_good['content'] != "non"]
len(df_good_new)

1125563

# Lọc bình luận tiêu cực
- Làm tương tự như bình luận tích cực

In [8]:
df_bad = df[df['rate']==1]
def zero(text):
    arr = text.split()
    if len(arr) >= min_words:
        return text
    else:
        return str('non')

df_bad['content'] = df_bad['content'].apply(lambda x: x.replace(
    str(x), zero(x)))

C:\Users\OnKhang\AppData\Local\Temp\ipykernel_5516\2587787443.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bad['content'] = df_bad['content'].apply(lambda x: x.replace(


### Số bình luận ít hơn max_words từ

In [9]:
print(len(df_bad[df_bad['content'] == "non"]))

9168


#### Số bình luận còn lại sau khi lọc

In [10]:
df_bad_new = df_bad[df_bad['content'] != "non"]
len(df_bad)

306361

# Trộn dữ liệu và Gộp 2 bảng lại

In [11]:
df_good_new

,content,rate
2,hàng chất_lượng giá rẻ hơn cửa_hàng nhưng bỉm có mùi hàng chất_lượng giá rẻ hơn cửa_hàng nhưng b...,0
8,bịch lớn nên dùng rất tiện không phải mua nhiều lần hầu_hết các cửa_hàng_không bán bịch to nên m...,0
9,mới đặt_hàng chiều muộn hôm trước sáng sớm hôm sau đã nhận được hàng quá nhanh quá bất_ngờ bỉm m...,0
10,lâu không dùng bobby ( trc trung_thành với bobby 4 năm ) giờ quay lại dùng thấy bỉm mỏng nhg độ ...,0
12,bỉm dùng tốt tuy_nhiên lớp chống tràn rất cứng mặc có vết lằn cho bé mỏng thấm hút nhanh hợp với...,0
...,...,...
1958536,giao hàng nhanh sản_phẩm tốt chất_lượng cao cửa_hàng phục_vụ tốt tiki chế_độ mã giảm_giá tốt,0
1958538,tranh giao hàng vào lúc nghĩ trưa 12 h,0
1958539,ổn lâu_dài không biết tn,0
1958540,quá là nét luôn hình_ảnh như thật luôn,0


In [12]:
df_bad_new

,content,rate
0,mua bỉm đợt sale 1 tr600k cho 4 bịch bỉm thì bị chia thành 2 đơn do hết hàng đợi vận_chuyển dài ...,1
1,quá tệ1 về giao hàng với chăm_sóc khách_hàng mình đồng_ý mùa dịch nên giao hàng có_thể chậm_trễ ...,1
3,chất_lượng bỉm thì tốt mà bên giao hàng và bên hỗ_trợ từ số hotline quá chậm chễ mình đặt 2 bịch...,1
4,hàng giả mọi người ơi đừng mua vì cái tã này mỏng hơn cái tã mình mua trực_tiếp của bobby luôn m...,1
5,nhận được hàng vui_lam nhưng có thắc_mắc xiu gia ghi là 816100 nhưng nhận vien giao lai thứ mình...,1
...,...,...
1958547,liên_hệ với tôi nhé máy_bay không tìm thấy chỗ để sạc pin,1
1958548,cửa_hàng cái đầu này cắm tn đăng_ký,1
1958549,tại_sao giao mình lại thiếu_phụ_kiện máy_ảnh vậy cửa_hàng,1
1958550,không có hướng_dẫn tiếng việt,1


### Lấy n bình luận tích cực và m bình luận tiêu cực

In [13]:
frame = [df_good_new[:max_data_good], df_bad_new[:max_data_bad]]
total = pd.concat(frame)
total = total.sample(frac=1)

In [14]:
total.shape

(14000, 2)

# Variable label & content

In [15]:
content = total['content']
labels = total['rate']
print(content.shape)
print(labels.shape)


(14000,)
(14000,)


# Import Bert
- Tiến hành đánh index từ bằng phoBert và padding dữ liệu cho dữ liệu có cùng chiều dài

In [16]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### Tokenized

In [17]:
tokenized = content.apply(lambda x: tokenizer.encode(
    x, max_length=max_len, truncation=True))

# Padding
- Padding để đảm bảo input có độ dài như nhau

In [18]:
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('padded:', padded[index])
print('len padded:', padded.shape)

padded: [   0  580 1685 3289  650  574  119 2164 3857    2    0    0    0    0
    0    0    0    0    0    0]
len padded: (14000, 20)


In [19]:
import pickle

def _save_pkl(path, obj):
  with open(path, 'wb') as f:
    pickle.dump(obj, f)


# Lưu lại các files
_save_pkl('./file_pkl/padding.pkl', padded)
_save_pkl('./file_pkl/labels.pkl', labels)
_save_pkl('./file_pkl/dataset.pkl', total)

### Mask
- Đánh dấu các từ được thêm vào, 1 là có từ, còn 0 thì bỏ qua
- Giúp mô hình bỏ qua các từ đc thêm vào ở padding
- Padding đc tạo để cho các input có số lượng đầu vào các từ bằng nhau


In [20]:
# get attention mask ( 0: not has word, 1: has word)
train_mask = np.where(padded == 0, 0, 1)

### Chuyển sang tensor
- Mô hình bert làm việc với dữ liệu dạng tensor nên phải chuyển về

In [21]:
train_text = torch.tensor(padded).to(torch.long)
train_mask = torch.tensor(train_mask)

In [27]:
# Check tập dữ liệu
total.tail(20)

,content,rate
33860,hi tikimình mua 2 hộp nhưng 1 hộp bị móp,1
2756,giao hàng nhanh gói hàng chắc_chắn,0
19752,không như quảng_cáo tệ quá,1
47147,hàng đóng_gói cẩn_thận nhưng giao hàng quá lâu 1 đơn hàng hơn tháng mới giao,1
12055,đóng gói sản_phẩm cẩn_thận,0
6367,hàng giao đúng ngàyđóng gói kĩ,0
51382,địa_chỉ giao hàng là hoàng ngọc hoà điện_thoại 0903775802 địa_chỉ 131 điện biên phủ phường 15 qu...,1
1149,sản_phẩm dùng tốt giá hợp_lý,0
12387,luc dat hang thi có hang khuyen_mai sao luc giao hang không có hang khuyen mai la sao yeu cau ti...,1
31986,mua 38 l mà giao 32 l không đúng như đã đặt,1


### Trích xuất đặc trưng và lưu vào file 

In [23]:
# Lấy features dầu ra từ BERT
with torch.no_grad():
    last_hidden_states = phobert(input_ids= train_text, attention_mask=train_mask)

v_features = last_hidden_states[0][:, 0, :].numpy()

In [28]:
v_features[0]

array([ 5.47165692e-01,  4.98807400e-01, -2.56964386e-01, -5.67711771e-01,
       -3.43030751e-01,  1.11693287e+00,  1.49377823e-01, -8.72437656e-01,
       -4.16315287e-01, -3.21063995e-01, -1.43329561e-01,  4.18511897e-01,
        1.89061403e-01,  1.21900767e-01, -5.41892350e-01, -1.98609978e-01,
       -1.68168291e-01,  1.84009314e-01, -1.23336688e-01,  2.24088103e-01,
        5.68099320e-03, -3.66659574e-02,  3.63412678e-01,  3.09004605e-01,
       -4.87409174e-01,  1.18755043e-01,  2.57364213e-01,  5.20214677e-01,
        4.41081405e-01, -2.34995604e-01, -3.01159471e-01,  4.70894933e-01,
        5.88273644e-01,  1.25807121e-01, -1.24412954e-01, -7.31444806e-02,
       -2.30305821e-01, -8.61031413e-02, -3.48356962e-02,  3.31719249e-01,
        1.23757616e-01,  4.28979397e-02, -2.94322580e-01, -1.40252143e-01,
        7.20985770e-01,  2.74496615e-01, -1.31095409e-01, -5.58155417e-01,
       -1.98988616e-01, -2.13066936e-02,  4.31698471e-01,  1.66676730e-01,
       -5.23333907e-01,  

In [29]:
v_features[0].shape

(768,)

In [24]:
# Lưu lại các files
_save_pkl('./file_pkl/v_features.pkl', v_features)